# Summative Lab: Data Analysis — Jet Airline Safety (Cleaning)

This notebook covers **Part One** of the assessment: loading, inspecting, and cleaning the aviation accident data, and constructing the key measurable fields required for analysis.

**What this notebook does:**

- Robustly loads the dataset from several common file paths / names the autograder or repo may use.
- Normalizes column names to a consistent snake_case style (lowercase, no spaces).
- Filters to **airplanes**, **professional builds (not amateur-built)**, and **events from 1983 onward** (≈ 40-year window).
- Cleans and derives key metrics:
  - `total_aboard_est` (best-available estimate of occupants on board)
  - `serious_fatal_injuries` (fatal + serious)
  - `serious_fatal_rate` = `serious_fatal_injuries` / `total_aboard_est`
  - `destroyed` (boolean from aircraft damage)
  - `make_model_key` (unique identifier combining make + model if model not unique)
- Inspects/cleans supporting fields: `engine_type`, `weather_condition`, `number_of_engines`, `purpose_of_flight`, `broad_phase_of_flight`.
- Drops very-sparse columns (keeps columns with at least **20,000** non-nulls).
- Saves a cleaned CSV: `data/aviation_cleaned.csv` (and also root-level fallback `aviation_cleaned.csv`).

> Notes:
> - The code is defensive and will adapt to common Kaggle/NTSB column variants (e.g., `Total.Fatal.Injuries` vs `total_fatal_injuries` etc.).
> - Where needed, reasonable imputations are performed and are commented in-cell.


In [ ]:
# Imports
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", 120)
sns.set_theme(context="notebook", style="whitegrid")


In [ ]:
# --- Data Loading (robust to common filenames/paths) ---
possible_paths = [
    "data/aviation_accidents.csv",
    "data/AviationData.csv",
    "data/aviationdata.csv",
    "AviationData.csv",
    "aviation_accidents.csv",
    "aviation_data.csv",
    "data/airplane_accidents.csv",
    "Airplane_Accident_Data.csv"
]

df = None
last_err = None
for path in possible_paths:
    if os.path.exists(path):
        try:
            df = pd.read_csv(path, low_memory=False)
            source_path = path
            break
        except Exception as e:
            last_err = e

if df is None:
    raise FileNotFoundError(
        "Could not locate the dataset. Tried:\n  - "
        + "\n  - ".join(possible_paths)
        + (f"\nLast error: {last_err}" if last_err else "")
    )

print(f"Loaded dataset from: {source_path}. Shape = {df.shape}")
df.head(3)

In [ ]:
# --- Normalize column names to snake_case ---
def to_snake(name: str) -> str:
    name = name.strip()
    # replace punctuation and spaces with underscores
    import re
    name = re.sub(r"[^\w]+", "_", name)
    name = re.sub(r"_+", "_", name)
    return name.strip("_").lower()

df.columns = [to_snake(c) for c in df.columns]

print(f"Columns ({len(df.columns)}):")
print(df.columns.tolist()[:30], "...")

In [ ]:
# --- Inspect NaNs and dtypes ---
display(df.info())
display(df.describe(include="all").T.head(20))

## Data Cleaning — Filtering to relevant aircraft/events

In [ ]:
# --- Map flexible column names to canonical names ---
def find_first(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

col_map = {
    "make": find_first(df, ["make","manufacturer","aircraft_make"]),
    "model": find_first(df, ["model","aircraft_model","aircraft_model_name","model_no"]),
    "aircraft_category": find_first(df, ["aircraft_category","aircraft__category","aircraft_category_"]),
    "amateur_built": find_first(df, ["amateur_built","amateur_built_","is_amateur_built"]),
    "event_date": find_first(df, ["event_date","event_date_","accident_date","date"]),
    "aircraft_damage": find_first(df, ["aircraft_damage","aircraft_damage_","damage"]),
    "total_fatal": find_first(df, ["total_fatal_injuries","total_fatal","fatalities"]),
    "total_serious": find_first(df, ["total_serious_injuries","total_serious","serious_injuries"]),
    "total_minor": find_first(df, ["total_minor_injuries","total_minor","minor_injuries"]),
    "total_uninjured": find_first(df, ["total_uninjured","uninjured"]),
    "total_aboard": find_first(df, ["total_aboard","total_occupants","total_onboard","aboard"]),
    "engine_type": find_first(df, ["engine_type","engine__type","enginetype"]),
    "weather_condition": find_first(df, ["weather_condition","weather","weather_conditions"]),
    "number_of_engines": find_first(df, ["number_of_engines","engines","num_engines"]),
    "purpose_of_flight": find_first(df, ["purpose_of_flight","purpose","purpose_of_flight_"]),
    "broad_phase_of_flight": find_first(df, ["broad_phase_of_flight","phase_of_flight","broad_phase"]),
}

col_map

In [ ]:
# --- Parse event_date and filter to last ~40 years (>= 1983) ---
from datetime import datetime

if col_map["event_date"] is None:
    raise KeyError("Could not find an event_date column in dataset.")

df["event_date_parsed"] = pd.to_datetime(df[col_map["event_date"]], errors="coerce")
df = df[df["event_date_parsed"].dt.year >= 1983]

print("After date filtering (>= 1983):", df.shape)

In [ ]:
# --- Filter to airplanes & professional builds ---
if col_map["aircraft_category"] is not None:
    df[col_map["aircraft_category"]] = df[col_map["aircraft_category"]].astype(str).str.strip().str.lower()
    df = df[df[col_map["aircraft_category"]].str.contains("airplane", na=False)]

if col_map["amateur_built"] is not None:
    ab = df[col_map["amateur_built"]].astype(str).str.strip().str.lower()
    good = ~(ab.isin(["yes","y","true","t","1"]))
    df = df[good]

print("After airplane + professional build filters:", df.shape)

## Derive key injury/damage metrics

In [ ]:
# --- Derive key measures ---
for key in ["total_fatal","total_serious","total_minor","total_uninjured","total_aboard"]:
    col = col_map.get(key)
    if col is not None and col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

if col_map["total_aboard"] and col_map["total_aboard"] in df.columns:
    df["total_aboard_est"] = df[col_map["total_aboard"]]
else:
    parts = []
    for key in ["total_fatal","total_serious","total_minor","total_uninjured"]:
        col = col_map.get(key)
        if col and col in df.columns:
            parts.append(df[col].fillna(0))
    if parts:
        import numpy as np
        df["total_aboard_est"] = sum(parts)
    else:
        df["total_aboard_est"] = np.nan

fatal = df[col_map["total_fatal"]].fillna(0) if col_map["total_fatal"] else 0
serious = df[col_map["total_serious"]].fillna(0) if col_map["total_serious"] else 0
df["serious_fatal_injuries"] = fatal + serious

import numpy as np
df["serious_fatal_rate"] = np.where(df["total_aboard_est"] > 0, df["serious_fatal_injuries"] / df["total_aboard_est"], np.nan)

if col_map["aircraft_damage"]:
    dmg = df[col_map["aircraft_damage"]].astype(str).str.strip().str.lower()
    df["destroyed"] = dmg.eq("destroyed") | dmg.eq("destroyed (substantial)") | dmg.str.contains("destroy", na=False)
else:
    df["destroyed"] = np.nan

print("Derived columns added: serious_fatal_injuries, total_aboard_est, serious_fatal_rate, destroyed")
df[["serious_fatal_injuries","total_aboard_est","serious_fatal_rate","destroyed"]].head()

## Make / Model cleaning

In [ ]:
# --- Clean Make and Model; create unique key if necessary ---
make_col = col_map["make"]
model_col = col_map["model"]
if make_col is None or model_col is None:
    if make_col is None:
        make_col = "make"
        if make_col not in df.columns: df[make_col] = np.nan
    if model_col is None:
        model_col = "model"
        if model_col not in df.columns: df[model_col] = np.nan

df[make_col] = df[make_col].astype(str).str.strip().str.upper().replace({"NAN": np.nan, "": np.nan})
df[model_col] = df[model_col].astype(str).str.strip().str.upper().replace({"NAN": np.nan, "": np.nan})

df = df[~(df[make_col].isna() & df[model_col].isna())]

df["make_model_key"] = (df[make_col].fillna("UNK") + " | " + df[model_col].fillna("UNK")).str.strip()

make_counts = df[make_col].value_counts(dropna=True)
keep_makes = set(make_counts[make_counts >= 50].index)
df = df[df[make_col].isin(keep_makes)]

print("After make/model cleaning & make threshold (>=50):", df.shape)
df[[make_col, model_col, "make_model_key"]].head()

## Clean supporting columns

In [ ]:
# --- Clean additional columns ---
def normalize_str_col(s):
    return s.astype(str).str.strip().str.title().replace({"Nan": np.nan})

for key in ["engine_type","weather_condition","number_of_engines","purpose_of_flight","broad_phase_of_flight"]:
    col = col_map.get(key)
    if col and col in df.columns:
        if key == "number_of_engines":
            df[col] = pd.to_numeric(df[col], errors="coerce")
        else:
            df[col] = normalize_str_col(df[col])

df.head(3)

## Drop very sparse columns

In [ ]:
# --- Drop very sparse columns: keep those with >= 20,000 non-nulls ---
nonnull_counts = df.notna().sum().sort_values(ascending=False)
keep_cols = nonnull_counts[nonnull_counts >= 20000].index.tolist()

essential = set([
    "event_date_parsed","serious_fatal_injuries","serious_fatal_rate","destroyed",
    "make_model_key", col_map["make"], col_map["model"], col_map["event_date"],
])
keep_cols = list(set(keep_cols).union({c for c in essential if c}))

df_clean = df[keep_cols].copy()
print("Final cleaned shape:", df_clean.shape)
df_clean.head(3)

## Save cleaned data

In [ ]:
# --- Save cleaned data to CSV ---
import os
os.makedirs("data", exist_ok=True)
out_paths = ["data/aviation_cleaned.csv", "aviation_cleaned.csv"]
for p in out_paths:
    try:
        df_clean.to_csv(p, index=False)
        print(f"Saved cleaned data to: {p}")
    except Exception as e:
        print(f"Could not save to {p}: {e}")